In [41]:
import os
import pandas as pd
import numpy as np
import re
import string

In [5]:
data_path = os.listdir('Datasets/Tweet Data')
data_path.sort()
data_path = ['Datasets/Tweet Data/{}'.format(file) for file in data_path]
data_path = pd.DataFrame(data_path,columns=['File Path'])

try:
    data_path = pd.concat([data_path,pd.DataFrame(data_path['File Path'].str.split(r'_',expand=True)[2])],axis=1)
    data_path = pd.concat([data_path,data_path[2].str.split(r'\.',expand=True)],axis=1,)
    data_path = data_path[['File Path',0]]
    data_path.columns = ['File Path', 'Counter']
    data_path['Counter'] = data_path['Counter'].apply(lambda x: int(x))
    
except:
    print('error')



In [6]:
for i,file in enumerate(data_path.values):
    if np.mod(file[1],1440)==0:
        print(i,file[1],pd.read_csv(file[0],index_col=0).shape)

0 1440 (19357, 11)


- Remove @[someone]
- Remove string 'RT'
- Remove punctuation (Remove # only; keep the string)
- Remove New Lines
- Remove links to http

In [76]:
def pre_process_tweet (text):
    #Remove all links starting with http...
    text = re.sub('https?:\/\/.*[\r\n]*','',text)
    #Remove RT
    text = re.sub('^RT[\s]+','',text)
    #Remove @[User]
    text = re.sub('@[^ ]+', '', text)
    #Remove Punctuation first
    text = text.translate(str.maketrans('','', string.punctuation))
    #Convert text to lower case
    text = text.lower()
    return text

In [77]:
unproc_data = pd.read_csv(data_path['File Path'][0],index_col=0)

In [78]:
print(unproc_data['text'][0])
pre_process_tweet(unproc_data['text'][0])  

RT @saylor: With USD prices increasing at the most rapid rate in 40 years, the @POTUS and @FederalReserve have both declared a "laser focus…


' with usd prices increasing at the most rapid rate in 40 years the  and  have both declared a laser focus…'